# Question 1
Given two strings s and t, determine whether some anagram of t is a substring of s. For example: if s = "udacity" and t = "ad", then the function returns True. Your function definition should look like: question1(s, t) and return a boolean True or False.

In [10]:
def question1(compare_string, ana_string, string_builder=""):
    """
    Given a string s, this function returns true if an anagram of t is a substring of s.
    Returns false otherwise.
    """
    #If compare string is empty, always return false
    if compare_string == "":
        return False
    
    #If no more characters to mutate
    if ana_string == "":
        #Always return false if string_builder is empty string, since "" in some_string
        #is true for all strings
        if string_builder == "":
            return False
        
        #Default case
        return string_builder in compare_string
    
    #Append chars recursively
    for i in range(len(ana_string)):
        #Get current char
        ch = ana_string[i]
        
        #Attempt recursive permutations
        curr_bool = question1(
            compare_string,
            ana_string[:i] + ana_string[i+1:],
            string_builder + ch
        )
        
        #Anagram found criteria
        if curr_bool:
            return True
    
    #No anagram found
    return False

### Explanation
This code runs in O(n! * m) time, where n is the length of the permutated string and m is the length of the string to find the anagram in. For a string with *n* characters, the only non-recursive (first) call runs in O(n) time, where a recursive call is made for each of the n characters. On the first recursive call for any character, the remaining *n*-1 characters will also undergo a recursive call, which continues until the remaining characters in the anagram string is 0. With this logic, *n* x *n*-1 * x ... x 2 x 1 gives n!. Searching within the comparer string requires searching over worst case *m* characters, so the time complexity is O(n!*m).

Because the only variable storage lies in the inputs, this algorithm requires O(1) space.

### Tests

In [11]:
q1_tests = [
    ("a", "a"),#True
    ("b", "a"),#False
    ("abcdefg", "ba"),#True
    ("abcdefg", "bad"),#False
    ("cinema", "iceman"),#True
    ("iceman52", "cinema")#True
]

for i in range(len(q1_tests)):
    print("Test {}: {}".format(i, question1(q1_tests[i][0], q1_tests[i][1])))

Test 0: True
Test 1: False
Test 2: True
Test 3: False
Test 4: True
Test 5: True


# Question 2
Given a string a, find the longest palindromic substring contained in a. Your function definition should look like question2(a), and return a string.

In [48]:
def question2(s):
    #Test base case
    if s == s[::-1]:
        return s
    
    best_pali = ""
    #Iterate over string
    for i in range(len(s)):
        #Attempt palindrome centering at current index
        curr_pali = s[i]
        left = i - 1
        right = i + 1
        
        #Iteratively build palindrome until broken
        while True:
            
            #Test both
            if -1 < left and right < len(s):
                both_pali = s[left] + curr_pali + s[right]
                if both_pali == both_pali[::-1]:
                    curr_pali = both_pali
                    left -= 1
                    right += 1
                    continue
                
            #Test left
            if -1 < left:
                left_pali = s[left] + curr_pali
                if left_pali == left_pali[::-1]:
                    curr_pali = left_pali
                    left -= 1
                    continue
                
            #Test right
            if right < len(s):
                right_pali = curr_pali + s[right]
                if right_pali == right_pali[::-1]:
                    curr_pali = right_pali
                    right += 1
                    continue
                
            #No better palindrome found
            break
                
        #Update if best pali has been beaten
        if len(best_pali) < len(curr_pali):
            best_pali = curr_pali
    return best_pali

### Explanation
This longest palindrome implementation attempts to build a long palindrome from each indexed character in the string *s*. At any given step the next-longest possible palindrome exists when the next left and right characters are both added to the string resulting in an increased length of 2. If such a palindrome doesn't exist, adding a leftward character and a rightward character are tested separately to see if a palindrome can still be grown. If not, the palindrome analysis stemming from the current index breaks.

Because this algorithm iterates over each of n characters, where n = len(s), with each while loop possibly iterating O(n) times, the runtime complexity is O(n^2). Additionally, the maximum storage used by the algorithm given at any time is O(n) (during string construction in the while loop), so the space requirement is linear.

### Tests

In [49]:
q2_tests = [
    ("", ""),
    ("a", "a"),
    ("ab", "a"),
    ("aba", "aba"),
    ("abcb", "bcb"),
    ("eat tae", "eat tae"),
    ("2442", "2442"),
    ("2244664432", "446644")
]

for i in range(len(q2_tests)):
    answer = question2(q2_tests[i][0])
    print("Test {}: question2({}) = {} ... expected {}".format(
        i, q2_tests[i][0], answer, q2_tests[i][1]
    ))

Test 0: question2() =  ... expected 
Test 1: question2(a) = a ... expected a
Test 2: question2(ab) = a ... expected a
Test 3: question2(aba) = aba ... expected aba
Test 4: question2(abcb) = bcb ... expected bcb
Test 5: question2(eat tae) = eat tae ... expected eat tae
Test 6: question2(2442) = 2442 ... expected 2442
Test 7: question2(2244664432) = 446644 ... expected 446644


# Question 3
Given an undirected graph G, find the minimum spanning tree within G. A minimum spanning tree connects all vertices in a graph with the smallest possible total weight of edges. Your function should take in and return an adjacency list structured like this:

{'A': [('B', 2)],

 'B': [('A', 2), ('C', 5)],
 
 'C': [('B', 5)]}
 
 Vertices are represented as unique strings. The function definition should be question3(G)

In [81]:
def question3(graph):
    """
    Given a graph of the form represnted in the above adjacency list, this function will
    return a minimum spanning tree for the graph in the same structure as the input.
    """
    
    #Return input if graph is empty
    if graph == None or len(graph) <= 1:
        return graph
    
    #Create subgraph with 1 vertex (vertex choice doesn't matter)
    min_vert = graph.keys()[0]
    sub = {min_vert: []}
    
    #Foreign edges graph, mapping foreign nodes to their least expense
    #connectable edge tuple (domestic vert, edge weight)
    #O(n)
    foreign = {}
    for tup in graph[min_vert]:
        foreign[tup[0]] = (min_vert, tup[1])
    #For each foreign key
    #O(n)
    while len(foreign) != 0:
        #Get minimum vertex
        min_graph_vert = ""
        min_edge_val = float("inf")
        min_sub_vert = ""
        
        #O(n)
        for graph_vert in foreign:
            sub_vert = foreign[graph_vert][0]
            edge_val = foreign[graph_vert][1]
            
            #If lower weight edge
            if edge_val < min_edge_val:
                #Store subgraph vertex
                min_sub_vert = sub_vert
                #Store foreign graph vertex
                min_graph_vert = graph_vert
                #Store edge weight
                min_edge_val = edge_val
        
        #Minimum edge found
        #Add edge entry for vertex already in the subgraph
        sub[min_sub_vert].append((min_graph_vert, min_edge_val))
        #Add entry for new foreign vertex
        sub[min_graph_vert] = [(min_sub_vert, min_edge_val)]
        
        #Remove graph vert from foreign (it's now in subgraph)
        del foreign[min_graph_vert]
        
        #Check if newly added vertex has better/new edges to offer
        #O(n)
        for tup in graph[min_graph_vert]:
            graph_vert = tup[0]
            graph_vert_edge_val = tup[1]
            
            #If identified vertex isn't already in subgraph
            if graph_vert not in sub:
                #If new foreign edge, add, or if better edge for foreign vertex, replace
                if graph_vert not in foreign or graph_vert_edge_val < foreign[graph_vert][1]:
                    foreign[graph_vert] = (min_graph_vert, graph_vert_edge_val)
    return sub

### Explanation
The code above is the unoptimized Prim's algorithm, which is used for finding minimum spanning trees. The algorithm starts by creating a subgraph containing 1 vertex with no outgoing nodes. From there, the graph maintains a map of foreign vertices (vertices not yet in the subgraph) as keys where the values are tuples containing the best connecting subgraph vertex and the corresponding edge weight between the subgraph and foreign vertices. During iteration, the least expensive edge from the foreign edges is chosen. Using this edge, the connecting foreign vertex is added to the subgraph. Additionally, if the new vertex connects to any undiscovered foreign nodes OR has a less expensive edge to connect to a currently known foreign known, such an edge is added or replaced, respectively.

This implementation runs in O(n^2) time, because the subgraph must collect all of the original n nodes, each of which could possibly offer a better n-1 edges in the case of a complete graph. The space complexity here is O(n) for the maintained subgraph adjacency list, which has 2x(edge count) total entries, but for the minimum spanning tree the edge count is exactly n-1 or O(n).

### Tests

In [80]:
q3_tests = [
    {
        'A': [('B', 2)],
        'B': [('A', 2), ('C', 5)],
        'C': [('B', 5)]
    },
    {},
    {"A": []},
    {
        'A': [('B', 2), ("C", 1)],
        'B': [('A', 2), ('C', 3)],
        'C': [("A", 1), ('B', 3)]
    },
    {
        'A': [('B', 2), ("C", 1)],
        'B': [('A', 2), ('C', 3), ("D", 4)],
        'C': [("A", 1), ('B', 3), ("D", 5)],
        "D": [("B", 4), ("C", 5)]
    }
]

q3_solns = [
    {
        'A': [('B', 2)],
        'B': [('A', 2), ('C', 5)],
        'C': [('B', 5)]
    },
    {},
    {"A": []},
    {
        'A': [('B', 2), ("C", 1)],
        'B': [('A', 2)],
        'C': [("A", 1)]
    },
    {
        'A': [('B', 2), ("C", 1)],
        'B': [('A', 2), ("D", 4)],
        'C': [("A", 1)],
        "D": [("B", 4)]
    }
]

#Test solutions
for i in range(len(q3_tests)):
    result = question3(q3_tests[i])
    
    #If correct number of vertices
    is_right = len(result) == len(q3_solns[i])
    if is_right:
        
        #Iterate over each vertex in each graph to assert set equality
        sets = [result, q3_solns[i]]
        for j in range(len(sets)):
            set_a = sets[j]
            set_b = sets[(j + 1) % len(sets)]
            for k in set_a:
                tups = set_a[k]
                for t in tups:
                    is_right = t in set_b[k]
                    if not is_right:
                        break
                if not is_right:
                    break
            if not is_right:
                break
    print("Test {} passed: {}".format(i, is_right))

Test 0 passed: True
Test 1 passed: True
Test 2 passed: True
Test 3 passed: True
Test 4 passed: True


# Question 4
Find the least common ancestor between two nodes on a binary search tree. The least common ancestor is the farthest node from the root that is an ancestor of both nodes. For example, the root is a common ancestor of all nodes on the tree, but if both nodes are descendents of the root's left child, then that left child might be the lowest common ancestor. You can assume that both nodes are in the tree, and the tree itself adheres to all BST properties. The function definition should look like question4(T, r, n1, n2), where T is the tree represented as a matrix, where the index of the list is equal to the integer stored in that node and a 1 represents a child node, r is a non-negative integer representing the root, and n1 and n2 are non-negative integers representing the two nodes in no particular order. For example, one test case might be

question4([[0, 1, 0, 0, 0],

           [0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0],
           [1, 0, 0, 0, 1],
           [0, 0, 0, 0, 0]],
          3,
          1,
          4)
          
and the answer would be 3.

**Note**: I'm pretty sure the input example provided is not valid input. There's 5 nodes in the tree but only 3 edges; there should be 4.

In [51]:
def question4(ll, root, n1, n2):
    """
    Returns the least common ancestor for the specified tree in ll rooted by the root indexed
    node in ll for child nodes n1 and n2.
    """
    n3 = root
    #Iterate until parent is found
    while True:
        
        left = -1
        right = -1
        #Get left and right child of n3
        for i in range(len(ll[n3])):
            if ll[n3][i] == 1:
                if ll[n3][i] < n3:
                    left = i
                elif n3 < ll[n3][i]:
                    right = i
                    break
        
        #Compare and set next node to left or right
        if n1 < n3 and n2 < n3:
            n3 = left
        elif n1 > n3 and n2 > n3:
            n3 = right
        #LCA found
        else:
            return n3
    #Bad
    return -1

This algorithm takes O(n) time to iterate over each element in a sublist out of *n* nodes in the tree. However, the height of the tree dictates how many iterations the n3 node is set to a child node, which can be accurately represented by h ~ log(n) for leaf nodes on a balanced BST. Therefore, runtime for this algorithm is O(nlog(n)), with O(1) storage.

### Tests

In [52]:
tests = [
    (
        [
            [0, 1, 0, 0, 0],
            [0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0],
            [1, 0, 0, 0, 1],
            [0, 0, 0, 0, 0]
        ],
        3,
        1,
        4
    ),
    (
        [
            [0, 0, 0, 0, 0],
            [1, 0, 0, 0, 0],
            [0, 1, 0, 1, 0],
            [0, 0, 0, 0, 1],
            [0, 0, 0, 0, 0]
        ],
        2,
        1,
        4
    ),
    (
        [
            [0, 1, 0],
            [0, 0, 1],
            [0, 0, 0]
        ],
        0,
        1,
        2
    )
]

solutions = [3, 2, 1]
for i in range(len(tests)):
    print("Test {} passed: {}".format(
        i,
        question4(tests[i][0], tests[i][1], tests[i][2], tests[i][3]) == solutions[i]
    ))

Test 0 passed: True
Test 1 passed: True
Test 2 passed: True


# Question 5
Find the element in a singly linked list that's m elements from the end. For example, if a linked list has 5 elements, the 3rd element from the end is the 3rd element. The function definition should look like question5(ll, m), where ll is the first node of a linked list and m is the "mth number from the end". You should copy/paste the Node class below to use as a representation of a node in the linked list. Return the value of the node at that position.

In [53]:
class Node:
    """
    Implementation of a singly linked list node.
    """
    def __init__(self, data):
        self.data = data
        self.next = None
        
#Note there's no need to implement a linked list class if I just need to iterate through
#a list of nodes. This problem is trivial
def make_fake_ll(data_list):
    """
    Given a list of the form [type1, type1, type1, ..., type1], this method returns a linked
    list of Node classes with the ith Node holding the ith type1 data and the ith Node's next
    points to a node holding the (i+1)th type1 data.
    """
    head = None
    #If list not empty, allocate head
    if 0 < len(data_list):
        head = Node(data_list[0])
    
    #Create nodes
    curr_node = head
    for i in range(1, len(data_list)):
        curr_node.next = Node(data_list[i])
        curr_node = curr_node.next
        
    #Return linked list
    return head

def question5(ll, m):
    """
    Given a linked list, referencable by a head node ll, this function returns the mth node
    from the end of the list. This method handles overflow for large m values s.t. they're
    bounded to the left by index 0. The element 17th-to-last in a 10 element list is therefore
    at index 0.
    """
    
    #Iterate once through to get length
    curr_node = ll
    length = 0
    while curr_node:
        length += 1
        curr_node = curr_node.next
    
    #Iterate until desired is found
    curr_node = ll
    diff = length - m
    while 0 < diff:
        diff -= 1
        curr_node = curr_node.next
    
    return curr_node

Note these tests take "Find the **element** in a singly linked list" literally. The node is returned and the data value is queried.

### Explanation
This analysis is just of the question5 function, not the linked list connector.

`question5()` runs in linear time. It iterates over the entire linked list at most 2 total times, with at least 1 complete iteration to get the length. Additionally, the algorithm only maintains pointers to the current node, so the space used is constant: O(1).

### Tests

In [58]:
#Tuples of the form (test linked list, test m, solution)
tests_and_solutions = [
    (make_fake_ll([0,1,2,3,4]), 4, 1),
    (make_fake_ll([]), 17, None),
    (make_fake_ll([i for i in range(27)]), 28, 0),
    (make_fake_ll([i-2 for i in range(27)]), 15, 10)
]

for i in range(len(tests_and_solutions)):
    tup = tests_and_solutions[i]
    answer = question5(tup[0], tup[1])
    if answer != None:
        answer = answer.data
    print("Test {} passed: {}".format(i, answer == tup[2]))

Test 0 passed: True
Test 1 passed: True
Test 2 passed: True
Test 3 passed: True
